### Load Francy Package

In [2]:
LoadPackage("francy");
LoadPackage("num");

#I  equal requirements in multiple declarations for operation `JUPYTER_ViewString'

#I  Please load package NormalizInterface or 4ti2Interface
#I  to have extended functionalities.
#I  Please load package SingularInterface or singular (not both)
#I  or GradedModules to have extended functionalities.


true

true

# Drawing Apéry sets

This example draws the Apéry set of a numerical semigrup with respect to its multiplicity. By passing over a node with the mouse, the set of factorizations with respect to the minimal generating system of the numerical semigroup is displayed. Clicking a node produces a message with the same information.

In [3]:
apery:=function(arg)
    local ap,c,hasse, s, n, r, graphHasse, aps, es, canvas, i, order, showfacts;
    # rel is a list of lists with two elements representin a binary relation
    # hasse(rel) removes from rel the pairs [x,y] such that there exists
    # z with [x,z],[z,y] in rel
    hasse:=function(rel)
      local dom, out;
      dom:=Flat(rel);
      out:=Filtered(rel, p-> ForAny(dom, x->([p[1],x] in rel) and ([x,p[2]] in rel)));
      return Difference(rel,out);
    end;

    order:=function(x)
        return Maximum(LengthsOfFactorizationsElementWRTNumericalSemigroup(x,s));
    end;

    showfacts:=function(x)
        Add(canvas, FrancyMessage(Concatenation(String(x), " factors as "),
                    String(FactorizationsElementWRTNumericalSemigroup(x,s))));
        return Draw(canvas);
    end;
    if Length(arg)=1 then
        s:=arg[1];
        n:=MultiplicityOfNumericalSemigroup(s);
    fi;
    if Length(arg)=2 then
        s:=arg[1];
        n:=arg[2];
    fi;
    if Length(arg)>2 then
        Error("The number of arguments must be one or two");
    fi;
  
    graphHasse := Graph(GraphType.HASSE);
    SetSimulation(graphHasse,true);
    SetDrag(graphHasse,true);
    ap:=AperyList(s,n);
    c:=Cartesian([1..n],[1..n]);
    c:=Filtered(c, p-> ap[p[2]]<>ap[p[1]]);
    c:=Filtered(c, p-> ap[p[1]]-ap[p[2]] in s);
    c:=hasse(c);
    aps:=[];
    for i in [1..n] do
        aps[i]:=Shape(ShapeType!.CIRCLE, String(ap[i]));
        SetLayer(aps[i],-order(ap[i]));
        Add(aps[i],Callback(showfacts,[ap[i]]));
        Add(aps[i],FrancyMessage(String(FactorizationsElementWRTNumericalSemigroup(ap[i],s))));
        Add(graphHasse,aps[i]);
    od;
    for r in c do
        Add(graphHasse,Link(aps[r[1]],aps[r[2]]));
    od;
    canvas:=Canvas("Apery");
    Add(canvas,graphHasse);
    return Draw(canvas);    
end;

function( arg... ) ... end

In [4]:
apery(NumericalSemigroup(10,51,27,31));

# Drawing sons of numerical semigroups

This example shows how to draw the sons of a numerical semigroup in the tree of numerical semigroups. If we click on a node, then the sets of sons of that node are added to the canvas, and if the node is a leaf, a warning message is displayed.

Passing the mouse over a node shows the set of minimal generators of the node.

In [5]:
sons:=function(s)
    local gens, frb, desc, graphHasse, d, shpr, shp, canvas, sonsf, i, gn, lbl;
    
    
    sonsf:=function(s,n,lb)
        local gens, frb, desc, d, shp, i, lbl, gn;

        frb:=FrobeniusNumber(s);
        gens:=Filtered(MinimalGenerators(s), x-> x>frb);
        desc:=List(gens, g->RemoveMinimalGeneratorFromNumericalSemigroup(g,s));
        gn:=Genus(s);
        i:=0;
        for d in desc do
            i:=i+1;
        lbl:=Concatenation(lb,":",String(i));
            shp:=Shape(ShapeType!.CIRCLE, lbl);
            SetLayer(shp,Genus(d));
            Add(shp,Callback(sonsf,[d,shp,lbl]));
            Add(shp,FrancyMessage(String(MinimalGenerators(d))));
            Add(graphHasse,shp);
            Add(graphHasse,Link(n,shp));
        od;
        if desc<>[] then 
            return Draw(canvas);
        fi;
        Add(canvas, FrancyMessage(FrancyMessageType.WARNING, "This semigroup is a leaf"));
        return Draw(canvas);
    end;
    
    frb:=FrobeniusNumber(s);
    gens:=Filtered(MinimalGenerators(s), x-> x>frb);
    desc:=List(gens, g->RemoveMinimalGeneratorFromNumericalSemigroup(g,s));
    gn:=Genus(s);

    graphHasse := Graph(GraphType.HASSE);
    shpr:=Shape(ShapeType!.CIRCLE, "S");
    Add(shpr,FrancyMessage(String(MinimalGenerators(s))));
    SetLayer(shpr,Genus(s));
    Add(graphHasse,shpr);
    i:=0;
    for d in desc do
        i:=i+1;
        lbl:=Concatenation("S",":",String(i));
        shp:=Shape(ShapeType!.CIRCLE, lbl);
        SetLayer(shp,Genus(d));
        Add(shp,Callback(sonsf,[d,shp,lbl]));
        Add(shp,FrancyMessage(String(MinimalGenerators(d))));
        Add(graphHasse,shp);
        Add(graphHasse,Link(shpr,shp));
    od;
    canvas:=Canvas("Sons of a numerical semigroup");
    Add(canvas,graphHasse);
    return Draw(canvas);    
end;

function( s ) ... end

In [6]:
sons(NumericalSemigroup(3,5,7));

# Tree of numerical semigroups

Now we draw the sons of a numerical semigroup `s` in the tree of numerical semigroups up to level `l`.

In [7]:
sonstree:=function(s,l)
    local gens, frb, desc, graphTreee, d, shpr, shp, canvas, sonsf;
    
    
    sonsf:=function(s,n,lv)
        local gens, frb, desc, d, shp;
        if lv=0 then 
            return ;
        fi;
        frb:=FrobeniusNumber(s);
        gens:=Filtered(MinimalGenerators(s), x-> x>frb);
        desc:=List(gens, g->RemoveMinimalGeneratorFromNumericalSemigroup(g,s));
        for d in desc do
            shp:=Shape(ShapeType!.CIRCLE, String(MinimalGenerators(d)));
            SetSize(shp,5);
            Add(graphTreee,shp);
            SetParentNode(shp,n);
            sonsf(d,shp,lv-1);
        od;
        if desc<>[] then 
            return ;
        fi;
        #Add(canvas, FrancyMessage(FrancyMessageType.WARNING, "This semigroup is a leaf"));
        return ;
    end;
    
    frb:=FrobeniusNumber(s);
    gens:=Filtered(MinimalGenerators(s), x-> x>frb);
    desc:=List(gens, g->RemoveMinimalGeneratorFromNumericalSemigroup(g,s));

    graphTreee := Graph(GraphType.TREE);
    SetCollapsed(graphTreee,false);
    shpr:=Shape(ShapeType!.CIRCLE, "S");
    SetSize(shpr,5);
    Add(shpr,FrancyMessage(String(MinimalGenerators(s))));
    Add(graphTreee,shpr);
    canvas:=Canvas("Sons of a numerical semigroup");
    Add(canvas,graphTreee);
    sonsf(s,shpr,l);
    return Draw(canvas);    
end;

function( s, l ) ... end

Darker dots correspond either to leaves or to elements with highest genus. Blue nodes can be collapsed by clicking.  

In [8]:
sonstree(NumericalSemigroup(3,5,7),4);